### doc_topics
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E TÓPICOS NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

In [69]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import pickle

In [70]:
outputs = "../outputs/"

In [79]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Seleciona textos da base mysql para fazer modelagem de tópicos
NOTA: DEIXEI EM RAW PORQUE ESSA EXTRAÇÃO TORNA O CÓDIGO MUITO LENTO, MAS DEVE SER FEITO DESSA FORMA NO CPU DO RENATO
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
texts = []
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()

with SSHTunnelForwarder(('200.20.164.147', 22),
                        #ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_username="marcelobribeiro",
                        ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           #user='marcelobribeiro', 
                           user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura documentos da tabela docs para posterior modelagem de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 1 AND readability != 'low' ") # filtra textos   
    data = cur.fetchall()
    numrows = cur.rowcount
    percentil = numrows/100
    
    for row in data:
        count += 1
        if row is None: break

        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')

        text =  row[1]
        text = text.split()
        symbols = [x for x in string.punctuation]
        text = [p for p in text if p not in symbols]
        text = [p.strip(string.punctuation) for p in text]
        text = [p for p in text if not p.isdigit()]
        text = [p for p in text if len(p)>1]
        texts.append(text)

········
········


In [15]:
# load nltk's English stopwords as variable called 'stopwords'

additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            additional_words

stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [85]:
#%time texts = [[word for word in text] for text in texts]
%time texts = [[word for word in text if word not in stopwords] for text in texts]

Wall time: 0 ns


In [86]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

126


In [110]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
#print(dictionary.token2id['']) #examina o id a partir do token
#print(dictionary.id2token[0]) #examina o token a partir do id
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [112]:
len(corpus)

10

#### Salva dictionary e corpus com o pickle

In [113]:
#filename = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/LDAcorpus.sav'
#filename = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/LDAdictionary.sav'
file_corpus = '../outputs/LDAcorpus.sav'
file_dictionary = '../outputs/LDAdictionary.sav'
pickle.dump(corpus, open(file_corpus, 'wb'))
pickle.dump(dictionary, open(file_dictionary, 'wb'))

### Teste com 20 tópicos.

In [91]:
%time lda = models.LdaModel(corpus, num_topics=20, id2word=dictionary, passes=50)

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
lda.print_topics(-1, num_words=5)

#### Visualizando os tópicos com o PyLDAvis (20 tópicos)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_20topics.html'))

### Teste com 30 tópicos.

In [54]:
%time lda = models.LdaModel(corpus, num_topics=30, id2word=dictionary, passes=50)

Wall time: 2.45 s


In [ ]:
lda.print_topics(-1, num_words=5)

#### Visualizando os tópicos com o PyLDAvis (30 tópicos)

In [ ]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_30topics.html'))

### Teste com 40 tópicos.

In [ ]:
%time lda = models.LdaModel(corpus, num_topics=40, id2word=dictionary, passes=50)

In [ ]:
lda.print_topics(-1, num_words=5)

#### Visualizando os tópicos com o PyLDAvis (40 tópicos)

In [ ]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [28]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_40topics.html'))

### Teste com 45 tópicos.

In [40]:
%time lda = models.LdaModel(corpus, num_topics=45, id2word=dictionary, passes=50)

Wall time: 2.64 s


In [ ]:
lda.print_topics(-1, num_words=5)

#### Visualizando os tópicos com o PyLDAvis (45 tópicos)

In [ ]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [28]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_45topics.html'))